# Text Cleaning Notebook


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
!pip install sematch

In [ ]:
!pip install gensim

In [ ]:
#importing libraries

import numpy as np
import pandas as pd
import itertools as it
import pickle
import glob
import os
import string
import gc
import re
import time
import nltk
import spacy
import textacy
import en_core_web_md
import sematch
import gensim

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from tqdm import tqdm, tqdm_notebook
from scipy import sparse
from scipy.optimize import minimize



In [ ]:
import numpy as np
import pandas as pd
import itertools as it
import pickle
import glob
import os
import string
import gc
import re
import nltk
import spacy
import textacy
import en_core_web_md
import sematch
from tqdm import tqdm
from nltk.corpus import wordnet as wn


# defining the cleaning functions
def text_to_wordlist(text):

    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "", text)
    text = re.sub(r"What's", "", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    return(text)

def concat_words(df2):
    df = df2.copy()
    df['Description'] = df['Description'].apply(lambda x: (' '.join(i for i in x)))
    return df

def basic_cleaning2(string):
    string = str(string)
    string = re.sub('[0-9\(\)\!\^\%\$\'\"\.;,-\?\{\}\[\]\\/]', ' ', string)
    string = re.sub(' +', ' ', string)
    return string

# simple cleaning 1
def clean_part1(df2):
    df = df2.copy()
    df['Description'] = df['Description'].apply(lambda x: text_to_wordlist(x))
    return df

# simple cleaning 2
def clean_part2(df2):
    df = df2.copy()
    df['Description'] = df['Description'].apply(lambda x: basic_cleaning2(x).split())
    df['Description'] = df['Description'].apply(lambda x: (' '.join(i for i in x)))
    return df



def words(text): return re.findall(r'\w+', text.lower())

def P(word): 
    return - WORDS[word] 

def correction(word): 
    try:
        corrected = max(candidates(word), key=P)
        return corrected
    except Exception:
        return word

def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# replaces mis spelled words
def known(words): 
    return set(w for w in words if w in WORDS)

# simple edits
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
#import tokenizer

nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize

In [ ]:
# main cleaning function

def clean_description():
    t = time.time()
    
    # reading train and test sets
    df_train = pd.read_csv('train.csv')
    df_test = pd.read_csv('test.csv')
    
    # setting nan in test and concatinating both
    df_test['Is_Response'] = np.nan
    df = pd.concat([df_train, df_test]).reset_index(drop=True)
    

    # perform basic cleanings
    df = clean_part1(df)
    df = clean_part2(df)
    
    # clean using textacy
    print('Cleaning using textacy.')
    df['Description'] = df['Description'].apply(lambda x: textacy.preprocess.preprocess_text(x, fix_unicode = True,
                                                                            lowercase = True,
                                                                            no_contractions = True,
                                                                            transliterate = True
                                                                            ))
    # simple leammtizations..
    print('Lemmatizing text.')
    SYMBOLS = set(' '.join(string.punctuation).split(' ') + ['...', '“', '”', '\'ve'])
    q1 = []
    for doc in tqdm(nlp.pipe(df['Description'], n_threads=2, batch_size=10000)):
        word_list = ([c.lemma_ for c in doc if c.lemma_ not in SYMBOLS])
        q1.append(' '.join(i for i in word_list))

    q1 = pd.DataFrame(q1)

    df['Description'] = q1

    # simple word corrections..
    print('Correcting words. Using most probable substitutes.')
    df['Description'] = df['Description'].apply(lambda x: (' '.join([correction(i) for i in word_tokenize(x)])))


    print('Text cleaning done, time it took:', time.time() - t)
    return df

In [ ]:
# download web_md data

nlp = en_core_web_md.load()

In [ ]:
# call function to clean the description
df = clean_description()

In [ ]:
# print the data frame
print(df.head(20))

In [ ]:
# save to csv
df.to_csv('alldata_cleaned.csv', index=False)